In [6]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import time

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import yaml

from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.max_rows', None)


In [3]:
with open('config.yaml', 'r') as f:
    config = yaml.load(f)

print(config)

{'PATH': {'PROJECT_PATH': 'D:\\Kaggle\\MachineLearning_CI_CD', 'SCRIPT_PATH': 'D:\\Kaggle\\MachineLearning_CI_CD\\scripts', 'MODELS_PATH': 'D:\\Kaggle\\MachineLearning_CI_CD\\models', 'DATA_PATH': 'D:\\Kaggle\\MachineLearning_CI_CD\\data', 'ANALYSIS_REPORTS_PATH': 'D:\\Kaggle\\MachineLearning_CI_CD\\documents\\Analysis\\'}}


In [5]:
X_train = pd.read_csv(os.path.join(config['PATH']['DATA_PATH'], 'X_train.csv'))
X_test = pd.read_csv(os.path.join(config['PATH']['DATA_PATH'], 'X_test.csv'))
print(X_train.head())
print('-----------------------------------------------------------------------')
print(X_test.head())

Id  SalePrice  MSSubClass  MSZoning  LotFrontage   LotArea  Street  \
0   931  12.211060    0.000000      0.75     0.461171  0.377048     1.0   
1   657  11.887931    0.000000      0.75     0.456066  0.399443     1.0   
2    46  12.675764    0.588235      0.75     0.394699  0.347082     1.0   
3  1349  12.278393    0.000000      0.75     0.388581  0.493677     1.0   
4    56  12.103486    0.000000      0.75     0.577658  0.402702     1.0   

   Alley  LotShape  LandContour  Utilities  LotConfig  LandSlope  \
0    1.0  0.333333     1.000000        1.0        0.0        0.0   
1    1.0  0.333333     0.333333        1.0        0.0        0.0   
2    1.0  0.000000     0.333333        1.0        0.0        0.0   
3    1.0  0.666667     0.666667        1.0        0.0        0.0   
4    1.0  0.333333     0.333333        1.0        0.0        0.0   

   Neighborhood  Condition1  Condition2  BldgType  HouseStyle  OverallQual  \
0      0.863636         0.4         1.0      0.75         0.6     0

In [7]:
## Compute the target
y_train = X_train['SalePrice']
y_test = X_test['SalePrice']

X_train.drop(['Id', 'SalePrice'], axis = 1, inplace = True)
X_test.drop(['Id', 'SalePrice'], axis = 1, inplace = True)

In [8]:
## Feature selection

sel_ = SelectFromModel(Lasso(alpha = 0.005, random_state = 0))
sel_.fit(X_train, y_train)

SelectFromModel(estimator=Lasso(alpha=0.005, copy_X=True, fit_intercept=True,
                                max_iter=1000, normalize=False, positive=False,
                                precompute=False, random_state=0,
                                selection='cyclic', tol=0.0001,
                                warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [9]:
sel_.get_support()

array([ True,  True, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False,  True,  True,
       False,  True,  True, False, False, False,  True, False, False,
       False, False,  True, False,  True, False, False, False, False,
       False, False, False,  True,  True, False,  True, False, False,
        True,  True, False, False, False, False, False,  True, False,
       False,  True,  True,  True, False,  True,  True, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [12]:
selected_feat = X_train.columns[(sel_.get_support())]
print(selected_feat)
print('Number of selected features:', len(selected_feat))

Index(['MSSubClass', 'MSZoning', 'Neighborhood', 'OverallQual', 'OverallCond',
       'YearRemodAdd', 'RoofStyle', 'MasVnrType', 'BsmtQual', 'BsmtExposure',
       'HeatingQC', 'CentralAir', '1stFlrSF', 'GrLivArea', 'BsmtFullBath',
       'KitchenQual', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageCars', 'PavedDrive'],
      dtype='object')
Number of selected features: 22


In [13]:
pd.Series(selected_feat).to_csv(os.path.join(config['PATH']['DATA_PATH'],'selected_features.csv'), index = False)